# SE3 Diffusion

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import nglview as nv

import numpy as np
import torch

from proteome import protein
from proteome.constants import residue_constants
from proteome.models.design.se3_diffusion import config
from proteome.models.design.se3_diffusion.modeling import SE3DiffusionForStructureDesign
from proteome.models.design.proteinmpnn.modeling import ProteinMPNNForSequenceDesign
from proteome.models.folding.omegafold.modeling import OmegaFoldForFolding

In [ ]:
structure_designer = SE3DiffusionForStructureDesign(model_name="paper", random_seed=0)

In [ ]:
designed_structure = structure_designer.design_structure(
    config.InferenceConfig(length=40)
)

In [ ]:
designed_pdb_str = protein.to_pdb(designed_structure)

In [ ]:
with open(f"reference_{structure_designer.model_name}.pdb", mode="w") as f:
    f.writelines(designed_pdb_str)

In [ ]:
view = nv.show_text(designed_pdb_str)
view

In [ ]:
sequence_designer = ProteinMPNNForSequenceDesign("ca_only_model-2", random_seed=37)
folder = OmegaFoldForFolding()

In [ ]:
sequence, score = sequence_designer.design_sequence(
    protein.to_ca_only_protein(designed_structure)
)
print(f"Sequence: {sequence} with global_score {score}")

In [ ]:
predicted_protein, confidence = folder.fold(sequence)
result_pdb = protein.to_pdb(predicted_protein)

In [ ]:
PLDDT_BANDS = [
  (0, 50, '#FF7D45'),
  (50, 70, '#FFDB13'),
  (70, 90, '#65CBF3'),
  (90, 100, '#0053D6')
]
view = py3Dmol.view(width=800, height=600)
view.addModelsAsFrames(result_pdb)

color_map = {i: bands[2] for i, bands in enumerate(PLDDT_BANDS)}
style = {'cartoon': {'colorscheme': {'prop': 'b', 'map': color_map}}}

style['stick'] = {}

view.setStyle({'model': -1}, style)
view.zoomTo()